In [2]:
# 필요한 라이브러리 불러오기
import os
import random
import argparse
from tqdm import tqdm
from datetime import datetime, timezone, timedelta
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
# 한전데이터 불러오기 

abs_path = 'C:/Users/crid2/kepcodata/'

train_data = pd.read_csv(abs_path + 'train.csv')
test_data = pd.read_csv(abs_path + 'test.csv')

In [15]:
# 시드 고정
RANDOM_SEED = 10
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# 경로 설정
ROOT_PATH = ''

# DATA_DIR = os.path.join(ROOT_PATH, 'data')
DATA_DIR = abs_path 

In [9]:
train_data

,index,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,...,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압,label_역률평균,label_전류고조파평균,label_전압고조파평균
0,0,2248930.50,28963.0,20237.0,59.854076,45.197918,259.916656,449.916656,47.5000,8663.00,...,260.750,452.750,10417.00,7304.00,48.71875,260.750,448.750,정상,경고,주의
1,1,0.00,0.0,0.0,0.000000,101.312500,0.000000,0.000000,24.3750,0.00,...,0.000,0.000,0.00,0.00,100.68750,0.000,0.000,경고,정상,정상
2,2,5375707.00,35244.0,19826.0,59.975650,107.385414,125.416664,216.750000,18.7500,11988.00,...,125.250,216.250,12236.00,6170.00,109.56250,125.000,216.250,정상,정상,주의
3,3,17781200.00,77056.0,39520.0,59.863000,244.854000,118.083000,205.333000,23.1250,25796.00,...,118.750,118.750,24992.00,13704.00,242.18800,118.000,118.000,정상,정상,경고
4,4,10143988.00,0.0,0.0,59.798140,0.000000,133.750000,231.500000,26.8750,0.00,...,134.500,231.500,0.00,0.00,0.00000,133.000,230.250,경고,정상,정상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420560,2420561,6229905.00,25925.5,22606.0,57.945000,52.833000,215.327000,372.958000,19.0630,8631.50,...,216.628,216.628,7818.50,7867.00,51.12500,215.128,215.128,주의,정상,경고
2420561,2420562,6057307.50,91244.0,47068.0,59.928820,152.156250,224.416672,388.583344,33.7500,30128.00,...,224.250,386.250,29052.00,14496.00,145.18750,223.250,388.750,정상,정상,주의
2420562,2420563,7966820.00,16822.0,11924.8,59.914000,31.363100,219.233000,379.862000,17.3214,5157.84,...,219.276,219.276,5954.98,4172.97,32.79530,219.560,219.560,정상,경고,경고
2420563,2420564,0.00,0.0,0.0,0.000000,44.572918,0.000000,0.000000,34.3750,0.00,...,0.000,0.000,0.00,0.00,41.96875,0.000,0.000,경고,경고,정상


In [10]:
sample_data = train_data.copy()

In [11]:
sample_data.head()

,index,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,...,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압,label_역률평균,label_전류고조파평균,label_전압고조파평균
0,0,2248930.5,28963.0,20237.0,59.854076,45.197918,259.916656,449.916656,47.500,8663.0,...,260.75,452.75,10417.0,7304.0,48.71875,260.75,448.75,정상,경고,주의
1,1,0.0,0.0,0.0,0.000000,101.312500,0.000000,0.000000,24.375,0.0,...,0.00,0.00,0.0,0.0,100.68750,0.00,0.00,경고,정상,정상
2,2,5375707.0,35244.0,19826.0,59.975650,107.385414,125.416664,216.750000,18.750,11988.0,...,125.25,216.25,12236.0,6170.0,109.56250,125.00,216.25,정상,정상,주의
3,3,17781200.0,77056.0,39520.0,59.863000,244.854000,118.083000,205.333000,23.125,25796.0,...,118.75,118.75,24992.0,13704.0,242.18800,118.00,118.00,정상,정상,경고
4,4,10143988.0,0.0,0.0,59.798140,0.000000,133.750000,231.500000,26.875,0.0,...,134.50,231.50,0.0,0.0,0.00000,133.00,230.25,경고,정상,정상


In [12]:
# 학습 데이터의 감정 분류 분포
sample_data[['index','label_전압고조파평균']].groupby('label_전압고조파평균').count().rename(columns={'':'count'})

,index
label_전압고조파평균,
경고,735155
정상,994194
주의,691216


In [53]:
# 데이터로드 파라미터
BATCH_SIZE = 2048
NUM_WORKERS = 0

In [14]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
import joblib

In [51]:
class CustomDataset(Dataset) : 
#             """ 필수 함수 : 
#             - __init__ : 초기화
#             - __len__ : 데이터셋(input)의 길이 반환
#             - __getitem__ : 데이터셋을 인덱스로 불러옴
#         """

#         """CustomDataset 클래스 정의

#         args:
#             data_dir (str)
#             mode (str)
#         """
    def __init__(self, data_dir, mode='train') : 
        self.mode = mode
        self.data_dir = data_dir
        
        # 테스트 모드 디코딩에 사용할 레이블 딕셔너리 
        self.label_dic = {0:'정상' , 1:'주의' , 2:'경고'}
        
        print('Loading' + self.mode + 'dataset..')
        
        # 데이터 경로가 존재하는지 확인
        if not os.path.isdir(self.data_dir) : 
            print(f'!!!, Cannot find {self.data_dir}....!!!')
            sys.exit()
            
        # csv파일로드 
        # val모드에서 train 데이터를 가져옵니다..? 
        
        if self.mode =='val' :
            self.data_path = os.path.join(self.data_dir, 'train.csv')
        else : 
            self.data_path = os.path.join(self.data_dir, f'{mode}.csv')
        
        df = pd.read_csv(self.data_path)
        
        
        # 학습/검증 데이터 구역선택 , 학습 데이터 비율에 따라 TRAIN_RATIO 변경
        TRAIN_RATIO = 0.9
        if self.mode == 'train':
            df = df[:int(len(df)*TRAIN_RATIO)]

        elif self.mode == 'val':
            df = df[int(len(df)*TRAIN_RATIO)+1:].reset_index(drop=True)

            
            
        if self.mode != 'test': 
            featrues = df.iloc[: , 1:-3].copy().fillna(0)
            targets = df.iloc[:  -3:].copy().fillna(0)
            
        else : 
            features = df.iloc[:, 1:].copy().fillna(0)
            
        feature_scaler = MinMaxScaler()
        feature_scaler_path = 'feature_scaler.pkl'
        
        if self.mode == 'train' : 
            feature_scaler.fit(features)
            joblib.dump(feature_scaler, feature_scaler_path)
        else : 
            feature_scaler = joblib.load(feature_scaler_path)
            
        self.features = feature_scalder.transform(features)
        
        
        
        # target encoding : 
        if self.mode != 'test' : 
            self.targets = targets.replace({
                '정상' : 0,
                '주의' : 1,
                '경고' : 2

            })
        
    def label_decoder(self, labels) : 
        try : 
            labels = list(mp(lambda x : self.label_dic[x] , labels))
            return labels
        except : 
            assert 'Invalid intent'
            
    
    def __len__(self) : 
        return self.features.shape[0]
    
    
    # 23개의 피쳐들과 test 모드일 때를 제외하고 target 세가지 를 리턴 
    
    def __getitem__(self, idx:int) : 
        
        feature = self.features[idx]  
        feature = torch.tensor(feature, dtype=torch.float)
        
        if self.mode != 'test' : 
            target_1 = int(self.targets.iloc[idx, 0])
            target_2 = int(self.targets.iloc[idx, 1])
            target_3 = int(self.targets.iloc[idx, 2])
            
        else : 
            return feature
        
        
        
    
            

In [ ]:
# DATASET 만들기 

train_dataset = CustomDataset(data_dir = DATA_DIR , mode = 'train')
validataion_dataset = CustomDataset(data_dri=DATA_DIR, mode='val')


In [ ]:
# DATASET 로딩하기 

train_dtaloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers = NUM_WORKERS, 
                             shuffle=True , drop_last = True)


In [7]:
#1. tartget 값을 encoder 라벨처리 

In [8]:
target_data = sample_data[['label_역률평균' ,'label_전류고조파평균','label_전압고조파평균']]
target_data

,label_역률평균,label_전류고조파평균,label_전압고조파평균
0,정상,경고,주의
1,경고,정상,정상
2,정상,정상,주의
3,정상,정상,경고
4,경고,정상,정상
...,...,...,...
2420560,주의,정상,경고
2420561,정상,정상,주의
2420562,정상,경고,경고
2420563,경고,경고,정상


In [9]:
# 결측값 확인 
sample_data.isnull().sum()

index               0
누적전력량              17
유효전력평균             17
무효전력평균             17
주파수                17
전류평균               17
상전압평균              17
선간전압평균             17
온도                 17
R상유효전력             18
R상무효전력             18
R상전류               18
R상전압               18
R상선간전압             18
S상유효전력           1380
S상무효전력           1380
S상전류             1380
S상전압             1380
S상선간전압           1380
T상유효전력           2898
T상무효전력           2898
T상전류             2898
T상전압             2898
T상선간전압           2898
label_역률평균          0
label_전류고조파평균       0
label_전압고조파평균       0
dtype: int64

In [10]:
sample_data.shape

(2420565, 27)

In [11]:
# 데이터 정규화 or 표준화 진행 
sample_data = sample_data.drop('index' ,axis=1)

In [12]:
# 결측값 대체 평균값으로 대체 #
# 0~23 결측값 평균으로 대체
for i in range(0, 23) : 
        sample_data.iloc[: , i] = sample_data.iloc[: , i].replace(np.NaN , sample_data.iloc[: , i].mean())

In [13]:
# 데이터값이 0인경에도 대체 
for i in range(0, 23) : 
        sample_data.iloc[: , i] = sample_data.iloc[: , i].replace(0 , sample_data.iloc[: , i].mean())

In [14]:
target_data = sample_data.iloc[: , -3:]

In [15]:
# target 값 정수화 함수 

def feature_encoder(df) :
    features = ['label_역률평균' , 'label_전류고조파평균' , 'label_전압고조파평균']
    
    for f in features :
        le = LabelEncoder()
        le = le.fit(df[f])
        df[f] = le.transform(df[f])
    
    return df 

In [16]:
target_data = feature_encoder(target_data)
target_data

,label_역률평균,label_전류고조파평균,label_전압고조파평균
0,1,0,2
1,0,1,1
2,1,1,2
3,1,1,0
4,0,1,1
...,...,...,...
2420560,2,1,0
2420561,1,1,2
2420562,1,0,0
2420563,0,0,1


In [17]:
# 0 : 경고 , 1 : 정상 , 2 : 주의

sample_data = feature_encoder(sample_data)
sample_columns = sample_data.columns
sample_columns 


Index(['누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균', '온도',
       'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압',
       'label_역률평균', 'label_전류고조파평균', 'label_전압고조파평균'],
      dtype='object')

In [18]:
# 결측값 재확인 
sample_data.isnull().sum()

누적전력량            0
유효전력평균           0
무효전력평균           0
주파수              0
전류평균             0
상전압평균            0
선간전압평균           0
온도               0
R상유효전력           0
R상무효전력           0
R상전류             0
R상전압             0
R상선간전압           0
S상유효전력           0
S상무효전력           0
S상전류             0
S상전압             0
S상선간전압           0
T상유효전력           0
T상무효전력           0
T상전류             0
T상전압             0
T상선간전압           0
label_역률평균       0
label_전류고조파평균    0
label_전압고조파평균    0
dtype: int64

In [19]:
#  정규화 standard scale 진행 
from sklearn.preprocessing import StandardScaler 

def Standard(df) : 
    scaler = StandardScaler()
    df = scaler.fit_transform(df.iloc[:, 0:23])
    
    return df 

In [20]:
sample_data = Standard(sample_data)

In [21]:
sample_data = pd.DataFrame(sample_data)
sample_data2 = sample_data.copy()

In [22]:
# 모델 돌려보기 

In [23]:
features = sample_data2 
target1 = target_data.iloc[: , 0]
target2 = target_data.iloc[: , 1]
target3 = target_data.iloc[: , 2]

In [24]:
X_train , X_test , y_train , y_test = train_test_split(features , 
                                                       target , 
                                                       test_size = 0.2 , 
                                                       random_state = 100)

X_train.shape , X_test.shape , y_train.shape , y_test.shape 

((1936452, 23), (484113, 23), (1936452, 3), (484113, 3))